In [ ]:
# 고우석
# torque/elbow_torque/1_팔꿈치.csv ~ 20_팔꿈치.csv 불러오고
# 값을 조금씩 조작해서 21.팔꿈치.csv ~ 40_팔꿈치.csv 저장

In [7]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
#공식을 위해 필요한 계산식 우선 계산 (M=86kg 으로 가정)
M=90+0.145
COM= (M*0.0162)*(0.2689*0.4574) / M*0.0162

#r의 값이 3가지라서 비슷한값을얻기위해 3가지경우 모두 고려
r1,r2,r3=0.266,0.255,0.116

# I_forarm_1=(M*0.0162)*(0.2689*r1)**2 #회전 방향 기준
# I_forarm_2=(M*0.0162)*(0.2689*r2)**2 #단면 기준
I_forarm_3=(M*0.0162)*(0.2689*r3)**2 # 길이방향기준이라서 팔꿈치의 I= i_3로 체택


#20회 동안의 MAX토크 데이터 저장
no_=[str(i) for i in range(1,21)]
tq=[]

In [9]:
# 우리가 촬영한 20개의 영상을 통해서 팔꿈치csv불러오기 (실제에선 공의 데이터를 불러오는 부분) 여기서 10, 11번파일은 오류때문에 csv측정이 잘되지않음 나머지는 정확함!
for j in range(1,21):
        df=pd.read_csv(f'torque/elbow_torque/{j}_팔꿈치.csv')
    
    
        # 여기서 속도와 가속도, 각속도 각가속도,토크항목을 제작.
        # 여기서 속도 초기에 0을 1번 넣는 이유
        #속도는 결국 위치의 변화량/시간 변화량 이므로, x1-x0/t1-t0 가 될수 밖에 없다. 위치값을 미분하면 속도인데 초기 위치 값은 상수이므로 v_0는 0이다
        #가속도는 결국 속도의 변화량/시간 변화량 이므로, v1-v0/t1-t0 가 될수 밖에 없다. 위치값을 미분했을때 상수를 미분해서 0이 한번 나오고, 상수인 속도를 미분해서 0이 한번 더 나오므로 a_0 와 a_1 0이다
    
        x_vel=[0]
        y_vel=[0]
        z_vel=[0]
        x_acc=[0,0]
        y_acc=[0,0]
        z_acc=[0,0]
    
        phi_vel=[0]
        theta_vel=[0]
        psi_vel=[0]
        phi_acc=[0,0]
        theta_acc=[0,0]
        psi_acc=[0,0]
    
        #토크값 계산후 넣을 리스트(절대값과 일반 둘 다)
        tq_z=[]
        tq_z_abs=[]
    
        # 좌표값, 각도값을 바탕으로 속도를 구하는 부분
        for i in range(1,df.shape[0]):
            x_vel.append((df['x'][i]-df['x'][i-1])/(df['time'][i]-df['time'][i-1]))
            y_vel.append((df['y'][i]-df['y'][i-1])/(df['time'][i]-df['time'][i-1]))
            z_vel.append((df['z'][i]-df['z'][i-1])/(df['time'][i]-df['time'][i-1]))
            phi_vel.append((df['phi'][i]-df['phi'][i-1])/(df['time'][i]-df['time'][i-1]))
            theta_vel.append((df['theta'][i]-df['theta'][i-1])/(df['time'][i]-df['time'][i-1]))
            psi_vel.append((df['psi'][i]-df['psi'][i-1])/(df['time'][i]-df['time'][i-1]))
    
        df['x_vel']=x_vel
        df['y_vel']=y_vel
        df['z_vel']=z_vel
        df['phi_vel']=phi_vel
        df['theta_vel']=theta_vel
        df['psi_vel']=psi_vel
    
        # 좌표값, 각도값을 바탕으로 가속도를 구하는 부분
        for i in range(2,df.shape[0]):
            x_acc.append((df['x_vel'][i]-df['x_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
            y_acc.append((df['y_vel'][i]-df['y_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
            z_acc.append((df['z_vel'][i]-df['z_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
            phi_acc.append((df['phi_vel'][i]-df['phi_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
            theta_acc.append((df['theta_vel'][i]-df['theta_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
            psi_acc.append((df['psi_vel'][i]-df['psi_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
    
        df['x_acc']=x_acc
        df['y_acc']=y_acc
        df['z_acc']=z_acc
        df['phi_acc']=phi_acc
        df['theta_acc']=theta_acc
        df['psi_acc']=psi_acc

In [10]:
df

,time,x,y,z,phi,theta,psi,x_vel,y_vel,z_vel,phi_vel,theta_vel,psi_vel,x_acc,y_acc,z_acc,phi_acc,theta_acc,psi_acc
0,0.000000,0.260498,0.948975,-0.272217,-0.011606,0.057421,-0.010385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.049805,0.287842,0.948486,-0.284180,0.021380,-0.061697,-0.063530,0.549021,-0.009804,-0.240196,0.662322,-2.391717,-1.067074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.099609,0.267822,0.948242,-0.280029,0.032376,-0.067195,-0.037263,-0.401960,-0.004903,0.083334,0.220774,-0.110387,0.527404,-19.094200,0.098407,6.495970,-8.865582,45.805524,32.014610
3,0.149414,0.253174,0.933838,-0.283447,0.045815,-0.062919,-0.003054,-0.294119,-0.289214,-0.068628,0.269834,0.085857,0.686852,2.165285,-5.708503,-3.051148,0.985057,3.940259,3.201454
4,0.199219,0.238281,0.920410,-0.261719,0.043371,-0.036652,-0.027489,-0.299018,-0.269607,0.436274,-0.049061,0.527404,-0.490609,-0.098369,0.393667,10.137640,-6.402916,8.865583,-23.641549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,40.192380,0.403320,0.913330,0.030273,0.043982,-0.045204,0.007941,-0.338267,0.348072,-0.308853,0.085865,0.858646,0.049065,-5.709883,7.875179,-8.170312,17.731298,27.584934,-23.641071
808,40.242190,0.394043,0.894287,0.047363,0.092852,0.061697,-0.006109,-0.186254,-0.382315,0.343101,0.981113,2.146184,-0.282070,3.051863,-14.663463,13.088801,17.973258,25.848988,-6.647969
809,40.291990,0.423584,0.912842,0.013672,-0.021380,-0.010385,-0.018937,0.593193,0.372586,-0.676534,-2.293811,-1.447432,-0.257594,15.651537,15.158657,-20.474593,-65.761524,-72.160956,0.491489
810,40.341800,0.420654,0.888916,0.001953,-0.025656,0.009774,0.004276,-0.058818,-0.480341,-0.235269,-0.085847,0.404709,0.466028,-13.089947,-17.123623,8.858964,44.327722,37.184114,14.527649


In [78]:
# 팔꿈치 더미데이터 생성
# 이경근은 딱 공을 20개만 던졌기 때문에, 60개의 더미데이터를 만들어야 한다
# 원본 데이터 읽기
df_origin = pd.read_csv('torque/elbow_torque/17_팔꿈치.csv')

# 원본 데이터셋의 평균 및 표준편차 계산
mean = df_origin.mean()
std = df_origin.std()

# 더미 데이터 생성
for j in list(range(10, 12)) + list(range(21, 81)):
    # 랜덤한 행의 수 생성
    num_rows = np.random.randint(700, 1100)

    # time 변수는 0.5초 간격으로 생성
    time = np.arange(0.5, 0.5 * (num_rows + 1), 0.5)

    # 각 요소에 대한 더미 데이터 생성
    x = np.interp(time, df_origin['time'], df_origin['x']) + np.random.normal(0, std['x'], size=num_rows)
    y = np.interp(time, df_origin['time'], df_origin['y']) + np.random.normal(0, std['y'], size=num_rows)
    z = np.interp(time, df_origin['time'], df_origin['z']) + np.random.normal(0, std['z'], size=num_rows)
    phi = np.interp(time, df_origin['time'], df_origin['phi']) + np.random.normal(0, std['phi'], size=num_rows)
    theta = np.interp(time, df_origin['time'], df_origin['theta']) + np.random.normal(0, std['theta'], size=num_rows)
    psi = np.interp(time, df_origin['time'], df_origin['psi']) + np.random.normal(0, std['psi'], size=num_rows)

    # 데이터프레임으로 변환
    df_dummy = pd.DataFrame({
        'time': time,
        'x': x,
        'y': y,
        'z': z,
        'phi': phi,
        'theta': theta,
        'psi': psi,
    })

    # 더미 데이터 프레임을 CSV 파일로 저장
    df_dummy.to_csv(f'torque/elbow_torque/{j}_팔꿈치.csv', index=False)

In [84]:
# 어깨 더미데이터 생성
# 이경근은 딱 공을 20개만 던졌기 때문에, 60개의 더미데이터를 만들어야 한다
# 원본 데이터 읽기
df_origin = pd.read_csv('torque/shoulder_torque/13_어깨.csv')

# 원본 데이터셋의 평균 및 표준편차 계산
mean = df_origin.mean()
std = df_origin.std()

# 더미 데이터 생성
for j in list(range(10, 12)) + list(range(21, 81)):
    # 랜덤한 행의 수 생성
    num_rows = np.random.randint(700, 1100)

    # time 변수는 0.5초 간격으로 생성
    time = np.arange(0.5, 0.5 * (num_rows + 1), 0.5)

    # 각 요소에 대한 더미 데이터 생성
    x = np.interp(time, df_origin['time'], df_origin['x']) + np.random.normal(0, std['x'], size=num_rows)
    y = np.interp(time, df_origin['time'], df_origin['y']) + np.random.normal(0, std['y'], size=num_rows)
    z = np.interp(time, df_origin['time'], df_origin['z']) + np.random.normal(0, std['z'], size=num_rows)
    phi = np.interp(time, df_origin['time'], df_origin['phi']) + np.random.normal(0, std['phi'], size=num_rows)
    theta = np.interp(time, df_origin['time'], df_origin['theta']) + np.random.normal(0, std['theta'], size=num_rows)
    psi = np.interp(time, df_origin['time'], df_origin['psi']) + np.random.normal(0, std['psi'], size=num_rows)

    # 데이터프레임으로 변환
    df_dummy = pd.DataFrame({
        'time': time,
        'x': x,
        'y': y,
        'z': z,
        'phi': phi,
        'theta': theta,
        'psi': psi,
    })

    # 더미 데이터 프레임을 CSV 파일로 저장
    df_dummy.to_csv(f'torque/shoulder_torque/{j}_어깨.csv', index=False)

In [80]:
#공식을 위해 필요한 계산식 우선 계산 (M=86kg 으로 가정)
M=90+0.145
COM= (M*0.0162)*(0.2689*0.4574) / M*0.0162

#r의 값이 3가지라서 비슷한값을얻기위해 3가지경우 모두 고려
r1,r2,r3=0.276,0.265,0.121

I_forarm_3=(M*0.0162)*(0.2689*r3)**2 # 길이방향기준이라서 팔꿈치의 I= i_3로 체택


#20회 동안의 MAX토크 데이터 저장
no_=[str(i) for i in range(1,21)]
tq=[]

# 우리가 촬영한 20개의 영상을 통해서 팔꿈치csv불러오기 (실제에선 공의 데이터를 불러오는 부분) 여기서 10, 11번파일은 오류때문에 csv측정이 잘되지않음 나머지는 정확함!
for j in range(21,41):
    df=pd.read_csv(f'torque/elbow_torque/{j}_팔꿈치.csv')


    # 여기서 속도와 가속도, 각속도 각가속도,토크항목을 제작.
    # 여기서 속도 초기에 0을 1번 넣는 이유
    #속도는 결국 위치의 변화량/시간 변화량 이므로, x1-x0/t1-t0 가 될수 밖에 없다. 위치값을 미분하면 속도인데 초기 위치 값은 상수이므로 v_0는 0이다
    #가속도는 결국 속도의 변화량/시간 변화량 이므로, v1-v0/t1-t0 가 될수 밖에 없다. 위치값을 미분했을때 상수를 미분해서 0이 한번 나오고, 상수인 속도를 미분해서 0이 한번 더 나오므로 a_0 와 a_1 0이다

    x_vel=[0]
    y_vel=[0]
    z_vel=[0]
    x_acc=[0,0]
    y_acc=[0,0]
    z_acc=[0,0]

    phi_vel=[0]
    theta_vel=[0]
    psi_vel=[0]
    phi_acc=[0,0]
    theta_acc=[0,0]
    psi_acc=[0,0]

    #토크값 계산후 넣을 리스트(절대값과 일반 둘 다)
    tq_z=[]
    tq_z_abs=[]

    # 좌표값, 각도값을 바탕으로 속도를 구하는 부분
    for i in range(1,df.shape[0]):
        x_vel.append((df['x'][i]-df['x'][i-1])/(df['time'][i]-df['time'][i-1]))
        y_vel.append((df['y'][i]-df['y'][i-1])/(df['time'][i]-df['time'][i-1]))
        z_vel.append((df['z'][i]-df['z'][i-1])/(df['time'][i]-df['time'][i-1]))
        phi_vel.append((df['phi'][i]-df['phi'][i-1])/(df['time'][i]-df['time'][i-1]))
        theta_vel.append((df['theta'][i]-df['theta'][i-1])/(df['time'][i]-df['time'][i-1]))
        psi_vel.append((df['psi'][i]-df['psi'][i-1])/(df['time'][i]-df['time'][i-1]))

    df['x_vel']=x_vel
    df['y_vel']=y_vel
    df['z_vel']=z_vel
    df['phi_vel']=phi_vel
    df['theta_vel']=theta_vel
    df['psi_vel']=psi_vel

    # 좌표값, 각도값을 바탕으로 가속도를 구하는 부분
    for i in range(2,df.shape[0]):
        x_acc.append((df['x_vel'][i]-df['x_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        y_acc.append((df['y_vel'][i]-df['y_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        z_acc.append((df['z_vel'][i]-df['z_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        phi_acc.append((df['phi_vel'][i]-df['phi_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        theta_acc.append((df['theta_vel'][i]-df['theta_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        psi_acc.append((df['psi_vel'][i]-df['psi_vel'][i-1])/(df['time'][i]-df['time'][i-1]))

    df['x_acc']=x_acc
    df['y_acc']=y_acc
    df['z_acc']=z_acc
    df['phi_acc']=phi_acc
    df['theta_acc']=theta_acc
    df['psi_acc']=psi_acc

    for i in range(df.shape[0]):
        a_elbow=y_acc[i]-psi_acc[i]*0.2689-psi_vel[i]*(psi_vel[i]*0.2689)-9.81 #팔꿈치부분에 부착했기때문에 축이 psi기준이다. 던질때 y방향으로 가장 빠르게 이동한다
        a_com=a_elbow+COM*(psi_acc[i]*0.2689)+psi_vel[i]*COM*(psi_vel[i]*0.2689)

        temp3=I_forarm_3 * psi_acc[i]+COM*(M*0.0162*a_com-M*0.0162*9.81)
        tq_z.append(-temp3*1600) # 선수데이터가 아니고, 던질때 센서가 확실히 고정되있지않아서 논문의 값 40~60사이값을 얻기 위해 보정
        tq_z_abs.append(abs(temp3*1600)) # 절대값을 통해 던지는 최대토크를 계산함. 보정치6

    df['tq_z']=tq_z
    df['tq_z_abs']=tq_z_abs
    tq.append(round((df['tq_z_abs'].max()),1)) 


In [81]:
#이상치 처리 84 이하 median값으로 보정  0=안전, 1=주의 2=위험
tq_e=pd.DataFrame(no_,columns=['회차'])
tq_e['elbow_Torque']=tq
tq_e.loc[tq_e['elbow_Torque']<84,['elbow_Torque']]=tq_e['elbow_Torque'].median()
tq_e['팔꿈치_부상위험']=0
tq_e.loc[tq_e['elbow_Torque']<105,['팔꿈치_부상위험']]=0
tq_e.loc[(tq_e['elbow_Torque']>=105)&(tq_e['elbow_Torque']<=119),['팔꿈치_부상위험']]=1
tq_e.loc[tq_e['elbow_Torque']>119,['팔꿈치_부상위험']]=2

In [82]:
tq_e

,회차,elbow_Torque,팔꿈치_부상위험
0,1,93.80,0
1,2,109.00,1
2,3,96.20,0
3,4,109.80,1
4,5,100.10,0
5,6,105.80,1
6,7,94.90,0
7,8,93.70,0
8,9,100.80,0
9,10,106.60,1


In [111]:
# 21~40.csv의 어깨 토크 대해 위의 코드를 반복

#공식을 위해 필요한 계산식 우선 계산 (M=86kg 으로 가정)
M=90+0.145
COM= (M*0.0271)*(0.2817*0.5772) / M*0.0271
#r의 값이 3가지라서 비슷한값을얻기위해 3가지경우 모두 고려
r1,r2,r3=0.285,0.269,0.158

I_forarm_3=(M*0.0271)*(0.2817*r3)**2 # 길이방향기준이라서 팔꿈치의 I= i_3로 체택


#20회 동안의 MAX토크 데이터 저장
no_=[str(i) for i in range(1,21)]
tq=[]

# 우리가 촬영한 20개의 영상을 통해서 팔꿈치csv불러오기 (실제에선 공의 데이터를 불러오는 부분) 여기서 10, 11번파일은 오류때문에 csv측정이 잘되지않음 나머지는 정확함!
for j in range(21,41):
    df=pd.read_csv(f'torque/shoulder_torque/{j}_어깨.csv')


    # 여기서 속도와 가속도, 각속도 각가속도,토크항목을 제작.
    # 여기서 속도 초기에 0을 1번 넣는 이유
    #속도는 결국 위치의 변화량/시간 변화량 이므로, x1-x0/t1-t0 가 될수 밖에 없다. 위치값을 미분하면 속도인데 초기 위치 값은 상수이므로 v_0는 0이다
    #가속도는 결국 속도의 변화량/시간 변화량 이므로, v1-v0/t1-t0 가 될수 밖에 없다. 위치값을 미분했을때 상수를 미분해서 0이 한번 나오고, 상수인 속도를 미분해서 0이 한번 더 나오므로 a_0 와 a_1 0이다

    x_vel=[0]
    y_vel=[0]
    z_vel=[0]
    x_acc=[0,0]
    y_acc=[0,0]
    z_acc=[0,0]

    phi_vel=[0]
    theta_vel=[0]
    psi_vel=[0]
    phi_acc=[0,0]
    theta_acc=[0,0]
    psi_acc=[0,0]

    #토크값 계산후 넣을 리스트(절대값과 일반 둘 다)
    tq_z=[]
    tq_z_abs=[]

    # 좌표값, 각도값을 바탕으로 속도를 구하는 부분
    for i in range(1,df.shape[0]):
        x_vel.append((df['x'][i]-df['x'][i-1])/(df['time'][i]-df['time'][i-1]))
        y_vel.append((df['y'][i]-df['y'][i-1])/(df['time'][i]-df['time'][i-1]))
        z_vel.append((df['z'][i]-df['z'][i-1])/(df['time'][i]-df['time'][i-1]))
        phi_vel.append((df['phi'][i]-df['phi'][i-1])/(df['time'][i]-df['time'][i-1]))
        theta_vel.append((df['theta'][i]-df['theta'][i-1])/(df['time'][i]-df['time'][i-1]))
        psi_vel.append((df['psi'][i]-df['psi'][i-1])/(df['time'][i]-df['time'][i-1]))

    df['x_vel']=x_vel
    df['y_vel']=y_vel
    df['z_vel']=z_vel
    df['phi_vel']=phi_vel
    df['theta_vel']=theta_vel
    df['psi_vel']=psi_vel

    # 좌표값, 각도값을 바탕으로 가속도를 구하는 부분
    for i in range(2,df.shape[0]):
        x_acc.append((df['x_vel'][i]-df['x_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        y_acc.append((df['y_vel'][i]-df['y_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        z_acc.append((df['z_vel'][i]-df['z_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        phi_acc.append((df['phi_vel'][i]-df['phi_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        theta_acc.append((df['theta_vel'][i]-df['theta_vel'][i-1])/(df['time'][i]-df['time'][i-1]))
        psi_acc.append((df['psi_vel'][i]-df['psi_vel'][i-1])/(df['time'][i]-df['time'][i-1]))

    df['x_acc']=x_acc
    df['y_acc']=y_acc
    df['z_acc']=z_acc
    df['phi_acc']=phi_acc
    df['theta_acc']=theta_acc
    df['psi_acc']=psi_acc

    for i in range(df.shape[0]):
        a_elbow=y_acc[i]-psi_acc[i]*0.2817-psi_vel[i]*(psi_vel[i]*0.2817)-9.81 #어깨부분에 부착했기때문에 축이 psi기준이다. 던질때 y방향으로 가장 빠르게 이동한다
        a_com=a_elbow+COM*(psi_acc[i]*0.2817)+psi_vel[i]*COM*(psi_vel[i]*0.2817)

        temp3=I_forarm_3 * psi_acc[i]+COM*(M*0.0271*a_com-M*0.0271*9.81)
        tq_z.append(-temp3*85) # 선수데이터가 아니고, 던질때 센서가 확실히 고정되있지않아서 논문의 값 100 이하값을 얻기 위해 보정
        tq_z_abs.append(abs(temp3*100)) # 절대값을 통해 던지는 최대토크를 계산함.

    df['tq_z']=tq_z
    df['tq_z_abs']=tq_z_abs
    tq.append(round((df['tq_z_abs'].max()),2))


In [112]:
#이상치 처리 100 이상 96값으로 보정
tq_s=pd.DataFrame(no_,columns=['회차'])
tq_s['shoulder_Torque']=tq
tq_s.loc[tq_s['shoulder_Torque']>100,['shoulder_Torque']]=96
tq_s['어깨_부상위험']=0
tq_s.loc[tq_s['shoulder_Torque']<30,['어깨_부상위험']]=1
tq_s

,회차,shoulder_Torque,어깨_부상위험
0,1,33.41,0
1,2,31.95,0
2,3,23.17,1
3,4,40.89,0
4,5,32.37,0
5,6,24.84,1
6,7,28.10,1
7,8,40.66,0
8,9,30.73,0
9,10,32.96,0


In [113]:
total=pd.merge(tq_e,tq_s,how='inner',on='회차')
total

,회차,elbow_Torque,팔꿈치_부상위험,shoulder_Torque,어깨_부상위험
0,1,93.80,0,33.41,0
1,2,109.00,1,31.95,0
2,3,96.20,0,23.17,1
3,4,109.80,1,40.89,0
4,5,100.10,0,32.37,0
5,6,105.80,1,24.84,1
6,7,94.90,0,28.10,1
7,8,93.70,0,40.66,0
8,9,100.80,0,30.73,0
9,10,106.60,1,32.96,0


In [114]:
total.to_csv('torque/wstorque.csv',index=False)